# Rayleigh-Ritz analysis of a 1D Elastic Bar

## Problem Statement  
Consider a unidimensional elastic bar of length `L`, with:  
- Linear density (mass per unit length): `ρ*A`  
- Axial stiffness: `E*A/L` (where `E` is Young’s modulus, `A` is cross-sectional area)  
- Displacement occurs only in the **x-direction**.  
- Boundary conditions (Dirichlet or Neumann) may be applied at the ends `x=0` and `x=L`.

In [7]:
import numpy as np

def F(x):
   # f1 = x**3 - x**2
   # f2 = x**4 - x**3
   f1 = x**3 - x**2 + 1
   f2 = np.cos(2*np.pi*x)-1
   f3 = x+1
   return np.array([f1, f2, f3])
   #return np.array([f1, f2])

Boundary Condition Compatibility Check

Write two Python functions to verify if a single-variable function satisfies given Dirichlet or Neumann boundary conditions at a point.

1. `is_dirichlet_compatible`
   - Parameters:
     - `fun` (callable): A function taking a single float argument.
     - `x_0` (float): Boundary point to evaluate.
     - `f_0` (float): Expected function value at the boundary point.
     - `etol` (float): Tolerance for equality check.
   - Returns:
     - `bool`: True if the function satisfies the Dirichlet condition at `x` within tolerance, False otherwise.

2. `is_neumann_compatible`
   - Parameters:
     - `fun` (callable): A function taking a single float argument.
     - `x_0` (float): Boundary point to evaluate.
     - `dfdx_0` (float): Expected derivative value at the boundary point.
     - `dtol` (float): Tolerance for derivative calculation. Assume `dx != 0`.
     - `etol` (float): Tolerance for equality check.
   - Returns:
     - `bool`: True if the function satisfies the Neumann condition at `x` within tolerance, False otherwise.

Implement these functions without external libraries.


**Example 1:**

```python
import numpy as np

def F(x):
    f1 = x**3
    f2 = np.exp(x) - 1
    return np.array([f1, f2])

print(is_dirichlet_compatible(F, x_0=0.0, f_0=0.0, etol=0.0001)) # u=0 @ x=0
print(is_neumann_compatible(F, x_0=0.0, dfdx_0=0.0, dtol=0.0001, etol=0.001)) # u'=0 @ x=0
```

*Output:*

```python
[ True  True]
[ True False]
```

**Example 2:**

```python
import numpy as np

def F(x):
    f1 = x**3 - x**2 + 1
    f2 = np.cos(2*np.pi*x)-1
    f3 = x+1
    return np.array([f1, f2, f3])

print(is_dirichlet_compatible(F, x_0=1.0, f_0=0.0, etol=0.0001)) # u=0 @ x=1
print(is_neumann_compatible(F, x_0=1.0, dfdx_0=1.0, dtol=0.0001, etol=0.001)) # u'=1 @ x=1
```

*Output:*

```python
[False  True False]
[ True False  True]
```

In [ ]:
import numpy as np

def is_dirichlet_compatible(fun, x_0, f_0, etol):
    """
    Verifies if a single-variable function satisfies a given Dirichlet boundary condition at a point.

    Parameters:
        fun (callable): A function taking a single float argument.
        x_0 (float): Boundary point to evaluate.
        f_0 (float): Expected function value at the boundary point.
        etol (float): Tolerance for equality check.

    Returns:
        bool: True if the function satisfies the Dirichlet condition at x within tolerance, False otherwise.
    """
    return np.abs(fun(x_0) - f_0) < etol

def is_neumann_compatible(fun, x_0, dfdx_0, dtol, etol):
    """
    Verifies if a single-variable function satisfies a given Neumann boundary condition at a point.

    Parameters:
        fun (callable): A function taking a single float argument.
        x_0 (float): Boundary point to evaluate.
        dfdx_0 (float): Expected derivative value at the boundary point.
        dtol (float): Tolerance for derivative calculation (dx). Assume dx != 0.
        etol (float): Tolerance for equality check.

    Returns:
        bool: True if the function satisfies the Neumann condition at x within tolerance, False otherwise.
    """
    # Approximate derivative using central difference
    # (f(x + dx) - f(x - dx)) / (2 * dx)
    # In this case, dtol is our dx
    derivative_approx = (fun(x_0 + dtol) - fun(x_0 - dtol)) / (2 * dtol)
    return np.abs(derivative_approx - dfdx_0) < etol    

## Governing Equations for a 1D Elastic Bar  

### **Strain Energy** (Potential Energy):  
$$  
V = \frac{1}{2} \int_{0}^{L} E A \left( \frac{\mathrm{d}u}{\mathrm{d}x} \right)^2 \mathrm{d}x  
$$  

### **Stiffness Matrix** (from Strain Energy):  
$$  
\mathbf{K} = \int_{0}^{L} E A \, \mathbf{F}'^\top \mathbf{F}' \, \mathrm{d}x  
$$  

### **Kinetic Energy**:  
$$  
T = \frac{1}{2} \int_{0}^{L} \rho A \dot{u}^2 \mathrm{d}x  
$$  

### **Mass Matrix** (from Kinetic Energy):  
$$  
\mathbf{M} = \int_{0}^{L} \rho A \, \mathbf{F}^\top \mathbf{F} \, \mathrm{d}x  
$$  

#### Notation:  
- $u$: Axial displacement  
- $\dot{u}$: Time derivative of displacement  
- $\mathbf{F}$: Vector of shape functions  
- $\mathbf{F}'$: Derivative of shape functions with respect to $x$  
- $E$: Young’s modulus, $A$: Cross-sectional area, $\rho$: Density  
- $L$: Length of the bar  

Write two Python functions to generate mass and stiffness matrices for a 1D finite element problem.

1. `generate_mass_matrix`
   - Parameters:
     - `shape_functions` (callable): Function providing shape functions and/or their properties. 
     - `length` (float): Length of the bar.
     - `linear_density` (float): Mass per unit length of the element.
     - `etol` (float): Tolerance for the numerical integration (should control the accuracy somehow).
   - Returns:
     - Mass matrix (numeric matrix representation).

2. `generate_stiffness_matrix`
   - Parameters:
     - `shape_functions` (callable): Function providing shape functions and/or their properties.
     - `length` (float): Length of the bar.
     - `axial_stiffness` (float): Young's modulus multiplied by cross-sectional area divided by the length.
     - `dtol` (float): Tolerance for derivative calculation.
     - `etol` (float): Tolerance for the numerical integration (should control the accuracy somehow).
   - Returns:
     - Stiffness matrix (numeric matrix representation).

Implement these functions using basic Python operations. Assume valid inputs are provided and the callable `shape_functions` returns all required geometric/functional information. 
Return matrices as a numpy arrays.

**Example 1:**

```python
length = 1
linear_density = 1 # rho*A
axial_stiffness = 1 # E*A/L

def F(x):
    f1 = x**3 - x**2
    f2 = x**4 - x**3
    return np.array([f1, f2])

print(generate_mass_matrix(F, length, linear_density, 1e-5), '\n')
print(generate_stiffness_matrix(F, length, axial_stiffness, 1e-5, 1e-5))
```

*Output:*

```python
[[0.00952388 0.00595242]
 [0.00595242 0.00396832]]
 
[[0.13333333 0.10000000]
 [0.10000000 0.0857143]]
```


**Example 2:**

```python
length = 2
linear_density = 10 # rho*A
axial_stiffness = 100 # E*A/L

def F(x):
    f1 = x**3 - x**2
    f2 = 3*x**4 - x**3
    f3 = x**2 - x
    return np.array([f1, f2, f3])

print(generate_mass_matrix(F, length, linear_density, 1e-3), '\n')
print(generate_stiffness_matrix(F, length, axial_stiffness, 1e-3, 1e-3))
```

*Output:*

```python
[[  33.52383859  335.23813163   18.66666667]
 [ 335.23813163 3382.85735747  185.90484911]
 [  18.66666667  185.90484911   10.66666667]] 

[[  4053.33334933  39360.00023199   1866.66667067]
 [ 39360.00023199 384548.57503839  17920.000076  ]
 [  1866.66667067  17920.000076      933.33333333]]
```

In [46]:
def generate_mass_matrix(f, length, linear_density, etol):
    # Calculate the width of each trapezoid
    n = int(round(1/etol)) #number of trapezoid
    h = (length - 0) / n

    # Generate the x-values for each point
    # x_values = [a + i * h for i in range(n + 1)] # List comprehension way
    x_values = np.linspace(0, length, n + 1) # More efficient NumPy way

    # Evaluate the function at each x-value
    y_values = [np.outer(f(x),f(x)) for x in x_values]

    # Apply the trapezoidal rule formula:
    # Integral approx = (h/2) * [f(x0) + 2f(x1) + 2f(x2) + ... + 2f(xn-1) + f(xn)]
    integral_sum = y_values[0] + y_values[-1]  # Add the first and last terms

    # Add the middle terms (multiplied by 2)
    for i in range(1, n):
        integral_sum += 2 * y_values[i]

    integral_approximation = (h / 2) * integral_sum

    return integral_approximation*linear_density


def generate_stiffness_matrix(f,length,axial_stiffness,dtol,etol):
    # Calculate the width of each trapezoid
    n = int(round(1/etol)) #number of trapezoid
    h = (length - 0) / n

    # Generate the x-values for each point
    # x_values = [a + i * h for i in range(n + 1)] # List comprehension way
    x_values = np.linspace(0, length, n + 1) # More efficient NumPy way

    # Evaluate the function at each x-value
    y_values = [np.outer((f(x+dtol)-f(x-dtol))/(2*dtol),(f(x+dtol)-f(x-dtol))/(2*dtol)) for x in x_values]

    # Apply the trapezoidal rule formula:
    # Integral approx = (h/2) * [f(x0) + 2f(x1) + 2f(x2) + ... + 2f(xn-1) + f(xn)]
    integral_sum = y_values[0] + y_values[-1]  # Add the first and last terms

    # Add the middle terms (multiplied by 2)
    for i in range(1, n):
        integral_sum += 2 * y_values[i]

    integral_approximation = (h / 2) * integral_sum

    return integral_approximation*axial_stiffness*length

In [47]:
length = 1
linear_density = 1 # rho*A
axial_stiffness = 1 # E*A/L

def F(x):
    f1 = x**3 - x**2
    f2 = x**4 - x**3
    return np.array([f1, f2])

print(generate_mass_matrix(F, length, linear_density, 1e-5), '\n')
print(generate_stiffness_matrix(F, length, axial_stiffness, 1e-5, 1e-5))

[[0.00952381 0.00595238]
 [0.00595238 0.00396825]] 

[[0.13333333 0.1       ]
 [0.1        0.08571429]]


In [48]:
length = 2
linear_density = 10 # rho*A
axial_stiffness = 100 # E*A/L

def F(x):
    f1 = x**3 - x**2
    f2 = 3*x**4 - x**3
    f3 = x**2 - x
    return np.array([f1, f2, f3])

print(generate_mass_matrix(F, length, linear_density, 1e-3), '\n')
print(generate_stiffness_matrix(F, length, axial_stiffness, 1e-3, 1e-3))


[[  33.52402286  335.2402819    18.66676   ]
 [ 335.2402819  3382.87954284  185.9057219 ]
 [  18.66676     185.9057219    10.66670667]] 

[[  4053.3456      39360.14959999   1866.67      ]
 [ 39360.14959999 384550.34486856  17920.0452    ]
 [  1866.67        17920.0452        933.3344    ]]
